<a href="https://colab.research.google.com/github/dlskawns/RecSys_and_Retrieval_Study/blob/main/%086_29_Collaborative_Filtering_memory_based.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Collaborative Filtering

### 정의

* 유저들의 선호도 또는 조회(1,0)를 바탕으로 특정 유저의 관심사를 예측한다. '집단지성'으로 다수의 의견을 반영하여, 유저 데이터가 많을 수록 정확해진다. 

* 유저 김씨와 비슷한 성향을 가진 유저들이 선호하는 아이템을 추천

* 아이템의 특성을 필요로 하지 않음

### 예시
* A 상품을 구매한 유저 김씨가 구매한 다른 상품들을 A 상품을 구해만 유저 박씨에게 추천

* 쿠팡의 예시: 
  * 다른 고객이 함께 구매한 상품
    * CB에서 Item Profile의 vector와는 다른 방식. CB의 Item Profile Vector는 사실상 Item 전부에 대한 Vector인 것. '다른 노트북 상품'과 같이 진행하는 것은 가능하다
  * 다른 고객이 함께 본 상품


### 목적
* 유저 u의 아이템 i에 대한 평점을 예측하는 것 (prediction)

### 방법
* 유저 - 아이템 행렬 생성
* 유사도 기준을 정하고, 유저 - 아이템 간 유사도 스코어 구하기
* 주어진 평점과 유사도를 활용하여 행렬의 비어있는 값(평점) 예측하기

### 특징
* CB보다도 구현이 간단하고
* CB보다도 성능도 안정적임
* 아이템이나 유저가 늘어날 때 확장성이 떨어진다.
* Sparse한 데이터일 경우 성능이 저하된다. 
  * sparse 한 경우는 사실 cold start와 비슷한 상황이라고도 볼 수 있다.
  * 너무 sparse한 경우, CB와의 하이브리드 추천을 이용해서 진행 할 수도 있다. 
  * Sparsity가 99.5%를 넘지 않도록 하는 것이 좋다고 함
  * 행렬의 전체 entry 가운데 비어있는 비율




## NeighborHood based 

### 1. User Based CF
두 유저가 얼마나 유사한 아이템을 선호하는가?  
  -> 특정 유저와 비슷한 유저가 있다면 그 유저가 좋아한 아이템을 추천할 수 있다.

||아이언맨|헐크|스타워즈|비포선라이즈|노팅힐|
|:-:|:-:|:-:|:-:|:-:|:-:|
|User A|5|4.5|5|2|1|
|User B|4|5|?|1|2|
|User C|2|1|1|4|5|
|User D|3|3|3|3|3|

-> 유저 A와 B의 유사도가 높다
  * 수학적으로 higly correlated

#### 1) Rating Prediction

Average(사용하지 않음): 각 유저간의 유사도를 구해 특정 아이템에 대한 평균 -> 매우 약한 성능

Weighted Average $\frac {\sum_usim(B,u)\cdot r_{u,i_n}} {\sum_nsim(B,u)}$: 유저 간의 유사도 값을 weight로 사용해 rating의 평균을 냄
  * User B 는 User A와 가장 비슷하므로(유사도가 높으므로) User A가 준 평점에 좀 더 힘이 실리게 된다. 

* 여기서 유저의 유사도는 아이템들에 대한 벡터를 만들어서 사용. 위의 경우는 각 유저가 5차원의 벡터로 표현이 됨

<br>

  






#### 2) 평점의 편향 해결 Deviation
모든 사람들이 기본적으로 주는 평점의 평균이 다 다르다. 
  * 누군가는 기본적으로 4\~5점을 줄 수 있고, 누군가는 기본적으로 2~3점으로 줄 수도 있음

* 편향을 해결하기 위해 절대평점을 사용하지 않음. 
  * 평균이 2.5점이 사람이 5점을 준 것은 매우 훌륭한 것. 
  * 반면 모든 아이템 평점을 5점으로 준 유저는 변별력이 떨어짐

* $dev(u,i) = r(u,i) - \bar r_{u'}$
  * dev = 아이템에 대한 평점 - 평균 평점
* $\hat r(u,i) = \bar{r_u}+ \frac {\sum_{u'\in\Omega_i}r(i',i)-\bar {r_u'}}{|\Omega_i|}$ = predicted rating = 유저평균 rating + dev rating

Weighted Average with deviation. 
* 이 경우도 가중 평균을 이용하는 것이 유저의 특징을 찾는 데엔 더욱 효과적일 수 있다.
* $\hat r(u,i) = \bar {r_u} + \frac {\sum \in \Omega_i sim(u,u')(r(u',i)-\bar{r_{u'}})}{}$

<br>

#### 3) K - Nearest Neighbors Collaborative Filtering

아이템 i에 대판 평점 예측을 위해 아이템 i에 대해 평가한 유저($\Omega_i$)의 데이터를 사용해야 한다. 
* KNN을 이용: 모든 유저활용 연산이 많고, 성능은 떨어지게 됨
  * 그렇기 떄문에 $\Omega_i$에 속한 User u와 가장 유사한 K명의 유저를 이용해 평점을 예측하는 것 (KNN)
  * 스타워즈를 본 사람 중에서 가장 유사한 사람을 25명 뽑아서 그 사람들의 가중평균을 이용한다.
  * 일반적으로 K는 25~50을 뽑아서 진행

<br>

### Item Based CF

아이템의 측면에서 유저의 평점을 예측한다는 개념으로 보면된다. 아이템 선호도를 바탕으로 연관성이 높은 다른 아이템을 추천한다. 

||아이언맨|헐크|스타워즈|비포선라이즈|노팅힐|
|:-:|:-:|:-:|:-:|:-:|:-:|
|User A|5|4.5|5|2|1|
|User B|4|5|?|1|2|
|User C|2|1|1|4|5|
|User D|3|3|3|3|3|

직관적으로 스타워츠와 아이언맨, 헐크의 유사도가 높고 비포선라이즈 및 노팅힐은 낮다.  
* 유저 B의 스타워즈에 대한 평점은 높을 것이다.
* 아이언맨(0.7), 헐크(0.9)가 유사도가 높아서 이 두가지를 바탕으로 예측한 것이다 K= 2
* $\frac {0.7\cdot 4 + 0.9 \cdot 5}{0.7+0.9} = 4.6$
* 여기서 유사도를 구하기 위한 벡터는 User based 때와는 달리 유저들의 리스트가 벡터가 된다. 

#### Rating Prediction
유저 u가 평가한 다른 아이템 중 아이템 i와 유사한 아이템들의 집합을 $\Phi_u$라고 할 때, 

* Average: $\hat r(u,i) = \frac {\sum_{i'\in\Phi_u}r(u,i')}{|\Phi_u|}$

* Weighted Average: $\hat r(u,i) = \frac {\sum_{i'\in\Phi_u}sim(i,i')r(u,i')}{\sum_{i'\in\Phi_u}(sim(i,i')}$

* 마찬가지로 아이템도 편향이 있을 수 있으므로 Deviation을 활용

  * Average: $\hat r(u,i) = \bar {r_i} + \frac {\sum_{i'\in\Phi_u}sim(i,i')(r(u,i')-\bar{r_i'})}{|\Phi_u|}$

  * Weighted Average: $\hat r(u,i) = \bar {r_i} + \frac {\sum_{i'\in\Phi_u}sim(i,i')(r(u,i')-\bar{r_i')}}{\sum_{i'\in\Phi_u}sum(i,i')}$
  * 유저 u가 평가를 많이 했을 경우에 해당 u가 본 영화들 중 i와 가장 비슷한 k개의 영화를 뽑은 것이 $\Phi_u$가 된다. 

유저가 많고, 아이템인 적은 경우  
User based CF
  * 구현이 쉽고 가 늘어날 수록 성능 저하
  * Item based 보다 다양한 추천 -> 비슷한 유저의 '다른 아이템'을 추천해주기 때문
  * Cold start에 취약
  * pearson 유사도를 사용할 때 성능이 높음

Item based CF
  * 보통 실제 서비스에서 User-based CF보다 높은 성능
  * 아이템 간 유사도 사용이 더 Robust
    * 아이템 기준 KNN이 더 많은 유저정보들로부터 고평가된(유사도가 높은) Neighborhood이기 때문에 User 기준보다 덜 변함
  * 추천에 대한 이유를 설명하기 훨씬 쉬움
    * 유저가 과거에 선호했던 다른 아이템과 비슷하기 때문에 추천
  * Cosine 유사도 사용시 성능이 높음 -> 실제로는 오프라인 테스트에선 모든 유사도 테스트를 함



#### Cold start
* 데이터가 충분하지 않으면 추천 성능이 떨어진다.
* 데이터가 전혀 없는 신규 유저나 아이템의 경우는 추천이 '불가능'
  * 인기도 기반 추천으로 휴리스틱한 스코어로 줄 수 있다. 

#### 계산 효율
* 유저와 아이템이 늘어날수록 유사도 계산이 증가 -> 매트릭스가 커짐
* 유저, 아이템이 많아야 정확한 예측을 하지만, 시간이 오래걸림

#### Long-tail shot
* 많은 유저들이 선호하는 소수 아이템이 추천 결과로 나타남 -> 인기도 기반으로 할때 높은 상품이 추천됨
* 롱테일에 포함되는 비주류 아이템이 추천되기는 어려움
* CF와 CB를 섞은 하이브리드로 이 단점을 해소


#### Similarity

1. Cosine Similarity
  * 주어진 두 벡터 X, Y에 대해 angle distance를 구하는 것
  * 비슷할 수록 유사도가 높아지며, 두 벡터 간의 각도가 좁아짐 -> 벡터간의 거리는 작아지는 것.
  * 두 벡터의 차원은 같아야 함
    * 벡터의 내적을 이용하는 방법이기 때문에 element wise하게 곱을 취해주기 위해선 두 벡터간의 차원이 같아야 함.
    * 같은 차원에 있어야 두 벡터가 임베딩 될 수 있음
  * $cos(\theta) = cos(X,Y) = \frac{X\cdot Y}{|X||Y|} = \frac {\sum_{i=1}^NX_i\cdot Y_i}{\sqrt{\sum_{i=1}^NX_i^2}\sqrt{\sum_{i=1}^NY_i^2}}$ 

2. Mean Squared Difference Similarity
  * 추천시스템에서 많이 사용하는 유사도
  * Binary가 아닌 실수형 데이터일 때(like 영화평점) 사용
  * 다른 유저 또는 아이템과의 유클리드 거리를 측정
    * 평점 - 평점을 활용하는 것으로 스케일이 맞아야 함(like 1~5).
  * 유저 $u$와 $v$의 유사도
    * $msd(u,v) = \frac {1}{|I_{uv}|}\cdot \displaystyle\sum_{i\in I_{ui}}(r_{ui}-r_{vi})^2$
    * $msd$_$sim(u,v) = \frac {1}{msd(u,v)+1}$
  * 아이템 $i$와 $j$의 유사도
    * $msd(i,j) = \frac {1}{|I_{ij}|}\cdot \displaystyle\sum_{i\in I_{ui}}(r_{ui}-r_{uj})^2$
    * $msd$_$sim(i,j) = \frac {1}{msd(i,j)+1}$
  * 분모에 1을 더하는 것은 smoothing 처리로 0이 되지 않도록 하는 것
    * 또한 유클리드 거리가 작을 수록 분모가 작아져 1에 가까워짐

3. Jaccard Similarity
  * 집합의 개념을 활용하는 방법으로 추천에서는 잘 활용하지 않는다.
  * 벡터의 차원 별 요소가 다르면 전혀 다른 벡터이지만, 자카드 유사도의 경우, 요소가 얼마나 겹치는지를 본다. 
    * 두 집합이 가진 아이템이 모두 같으면 1 하나도 없으면 0
  * J(A,B)= $\frac {|A\cap B|}{|A \cup B|} = \frac {|A \cap B|}{|A|+|B|-|A \cap B|}$

4. Pearson Similarity (Pearson Correlation)
  * 피어슨 상관도를 구한다고 보면 됨 (상관계수)
  * 코사인 유사도와 비슷하지만, X, Y 벡터를 표본 평균으로 정규화 한 뒤에 코사인 유사도를 구한 값
  * 직관적 해석으로는 X와 Y가 함께 변하는 정도 / X와 Y가 따로 변하는 정도
  * $pearson$_$sim(X,Y) = \frac {\sum_{i=1}^N(X_i-\bar X)(Y_i-\bar Y)}{\sqrt{\sum_{i=1}^N(X_i-\bar X)^2}\sqrt{\sum_{i=1}^N(Y_i-\bar Y)^2}}$
  * 유저 - 아이템이 아래와 같이 평점이 결정된 경우, 코사인 유사도는 좀 더 낮게 나오는 반면, 피어슨 유사도는 정규화로 인해 유저의 편향을 고려하게 되어 유사도가 높게 나온다. 
    * 이러한 이유로 user based CF에선 피어슨 유사도가 보통 더 잘 나온다고 한다.

||$i_1$|$i_2$|$i_3$|
|:-:|:-:|:-:|:-:|
|$u_1$|1|2|3|
|$u_2$|4|5|6|

